# Setup

In [2]:
!pip install datasets transformers torch accelerate sacremoses sacrebleu --quiet
!pip install dataset --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.53 which is incompatible.


In [3]:
import torch, os
from transformers import MarianTokenizer, MarianMTModel
import pandas as pd
import sacrebleu
from tqdm import tqdm
from datasets import load_dataset


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["WANDB_DISABLED"] = "true"

# Load the Model and Tokenizer

In [5]:
model_name = '/kaggle/input/finetuned-opusmt-en-to-hi-model'
tokenizer = MarianTokenizer.from_pretrained(model_name)
# Load the model
model = MarianMTModel.from_pretrained(model_name)
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(64110, 512, padding_idx=64109)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(64110, 512, padding_idx=64109)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

# Tatoeba Benchmark Evaluation

## Load Tatoeba dataset

In [6]:
# Load the dataset 
df = pd.read_csv('/kaggle/input/tatoeba-hindi/Tatoeba-Hindi.csv')
english_sentences = df['English'].tolist()
hindi_sentences = df['Hindi'].to_list()

In [7]:
print(english_sentences[:10])
print(hindi_sentences[:10])

["A baby is God's opinion that the world should go on.", 'Absence of rain caused the plants to die.', 'A button has come off my raincoat.', 'A cat ran after a mouse.', 'A clock has two hands.', 'A country is a dangerous machine.', 'A crow is as black as coal.', 'Actinium was discovered by André-Louis Debierne in 1899.', 'Add a little sugar and cream.', 'Adopt the pace of nature: her secret is patience.']
['नन्हे शिशु के जन्म का अर्थ है कि भगवान यह चाहते हैं कि यह दुनिया बनी रहे।', 'पौधे बारिश के बिना मर गए।', 'मेरे रेनकोट से एक बटन निकल आया है।', 'एक बिल्ली चूहे के पीछे भागी।', 'घड़ी के दो हाथ होते हैं.', 'देश एक खतरनाक मशीन होती है।', 'कौआ कोयले जैसा काला होता है।', 'ऐक्टिनियम का खोज आंड्रे-लूई डेबिएर्न ने साल १८९९ में किया था।', 'थोड़ी शक़्क़र और मलाई डालो।', 'प्रकृति की गति अपनाएं: उसका रहस्य है धीरज।']


## Calculate BLEU: Tatoeba

In [8]:
# Function to generate translation for a given input text
def generate_translation(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        output_ids = model.generate(input_ids)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

translations = []
references = []
for i in tqdm(range(0, len(english_sentences)), desc="Translating"):
    translations.append(generate_translation(english_sentences[i]))
    references.append([hindi_sentences[i]])

# Calculate BLEU score
bleu = sacrebleu.corpus_bleu(translations, references)
print(f"BLEU score: {bleu.score}")


Translating: 100%|██████████| 5000/5000 [46:25<00:00,  1.80it/s]  


BLEU score: 12.334713408204749


# IN-22 Benchmark Evaluation

## Load IN-22 dataset

In [9]:
# download and load specific pairs
dataset = load_dataset("ai4bharat/IN22-Gen", "eng_Latn-hin_Deva")


The repository for ai4bharat/IN22-Gen contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ai4bharat/IN22-Gen.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating gen split: 0 examples [00:00, ? examples/s]

In [10]:
dataset

DatasetDict({
    gen: Dataset({
        features: ['id', 'context', 'source', 'url', 'domain', 'num_words', 'bucket', 'sentence_eng_Latn', 'sentence_hin_Deva'],
        num_rows: 1024
    })
})

In [11]:
english_sentences = dataset['gen']['sentence_eng_Latn']
hindi_sentences = dataset['gen']['sentence_hin_Deva']

# Convert them to lists
english_sentences = list(english_sentences)
hindi_sentences = list(hindi_sentences)

# Verify the first few elements of each list
print(english_sentences[:5])
print(hindi_sentences[:5])


['An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.', 'Ajanta, located in the Aurangabad District of Maharashtra has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.', 'Body colour gets merged with the outer line, creating the effect of volume.', 'Ashoka started making extensive use of stone for sculptures and great monuments, whereas the previous tradition consisted of working with wood and clay.', 'Potatoes mixed in masalas, coated in besan batter and deep fried to perfection form this delicious and famous dish of Maharashtra.']
['सेवा संबंधी लोगों के लिए भेष कई गुणों का संयोजन है, जैसे कि उनके जूते, कपड़े, टाई, आभूषण, केश शैली, मेक-अप, घड़ी, कॉस्मेटिक, इत्र, आदि।', 'महाराष्ट्र के औरंगाबाद जिले में स्थित अजंता में उन्तीस चैत्य और विहार गुफाएँ हैं जो पहली शताब्दी ई.पू. से ले कर पाँचवीं शताब

In [15]:
from transformers import MarianTokenizer, MarianMTModel

model_name = '/kaggle/input/finetuned-opusmt-en-to-hi-model'
tokenizer = MarianTokenizer.from_pretrained(model_name)
# Load the model
model = MarianMTModel.from_pretrained(model_name)
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(64110, 512, padding_idx=64109)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(64110, 512, padding_idx=64109)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

## Calculate BLEU: IN-22

In [16]:
# Function to generate translation for a given input text
def generate_translation(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        output_ids = model.generate(input_ids)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

translations = []
references = []
for i in tqdm(range(0, len(english_sentences)), desc="Translating"):
    translations.append(generate_translation(english_sentences[i]))
    references.append([hindi_sentences[i]])

# Calculate BLEU score
bleu = sacrebleu.corpus_bleu(translations, references)
print(f"BLEU score: {bleu.score}")


Translating: 100%|██████████| 1024/1024 [47:56<00:00,  2.81s/it]


BLEU score: 26.009600942848696


## Calculate chrF

In [14]:
translations = []
references = []
for i in tqdm(range(0, len(english_sentences)), desc="Translating"):
    translations.append(generate_translation(english_sentences[i]))
    references.append(hindi_sentences[i])  
# Calculate chrF score
chrf = sacrebleu.corpus_chrf(translations, references)
print(f"chrF score: {chrf.score}")

Translating: 100%|██████████| 1024/1024 [40:38<00:00,  2.38s/it]


chrF score: 4.633715798764343
